In [1]:
import Data.List
import Database.HDBC.Sqlite3
import Database.HDBC
import Numeric.LinearAlgebra.Data
import Numeric.LinearAlgebra.HMatrix

In [2]:
db <- connectSqlite3 "../data/movies.sqlite3"

In [3]:
results <- quickQuery db "SELECT IFNULL(data.rating, 0) FROM numberview LEFT OUTER JOIN data ON (numberview.userid=data.userid AND numberview.itemid=data.itemid)" []

In [4]:
length results

1586126

In [5]:
values = map fromSql (head (transpose results)) :: [Double]

In [6]:
m = matrix 1682 values

In [7]:
size m

(943,1682)

In [8]:
(mean, cov) = meanCov m

In [9]:
size cov

(1682,1682)

In [10]:
(evalues, evectors) = eigSH cov

In [11]:
size evectors

(1682,1682)

In [12]:
size evalues

1682

In [13]:
sum (toList evalues)

1193.7523983852445

In [14]:
sum (take 1 (toList evalues))

201.8774412573846

In [15]:
sum (take 2 (toList evalues))

258.70043064565687

In [16]:
sum (take 1682 (toList evalues))

1193.7523983852445

In [17]:
sum (take 50 (toList evalues))

628.1658651602687

In [18]:
sum (take 200 (toList evalues))

967.6354827452101

In [19]:
r = subMatrix (0,0) (1682, 200) evectors

In [20]:
size r

(1682,200)

In [21]:
mprime = mul m r

In [22]:
size mprime

(943,200)

In [23]:
quickQuery db "SELECT * FROM users WHERE userid=11" []

[[SqlInt64 11,SqlInt64 39,SqlByteString "F",SqlByteString "other",SqlInt64 30329]]

In [24]:
quickQuery db "SELECT title FROM items,data WHERE data.userid=11 AND data.itemid=items.movieid AND data.rating >= 4" []

[[SqlByteString "Truth About Cats & Dogs, The (1996)"],[SqlByteString "Bob Roberts (1992)"],[SqlByteString "Jane Eyre (1996)"],[SqlByteString "Boys on the Side (1995)"],[SqlByteString "Last Supper, The (1995)"],[SqlByteString "Real Genius (1985)"],[SqlByteString "Carrington (1995)"],[SqlByteString "Star Trek IV: The Voyage Home (1986)"],[SqlByteString "Midnight in the Garden of Good and Evil (1997)"],[SqlByteString "Harold and Maude (1971)"],[SqlByteString "Day the Earth Stood Still, The (1951)"],[SqlByteString "Dances with Wolves (1990)"],[SqlByteString "Unforgiven (1992)"],[SqlByteString "Ed Wood (1994)"],[SqlByteString "Ghost (1990)"],[SqlByteString "Butch Cassidy and the Sundance Kid (1969)"],[SqlByteString "Shadowlands (1993)"],[SqlByteString "2001: A Space Odyssey (1968)"],[SqlByteString "Chasing Amy (1997)"],[SqlByteString "Michael Collins (1996)"],[SqlByteString "Dead Man Walking (1995)"],[SqlByteString "Psycho (1960)"],[SqlByteString "Rosencrantz and Guildenstern Are Dead (199

In [25]:
copy = fromRows $ replicate 943 (mprime ! 10)

In [26]:
size copy

(943,200)

In [27]:
diffSqrd = (mprime - copy) ^ 2

In [28]:
difflist = map (\i -> sum (toList (diffSqrd ! i) :: [Double])) [0..942]

In [29]:
take 20 difflist

[3888.06887804032,2421.7564163988777,2195.4243195280765,2098.959655710307,2026.6558331454823,3385.08651584399,5695.452965754604,2285.5064935671694,2168.355976327543,3895.765079969442,0.0,2209.0389625422945,6201.65247199578,2550.9518870160455,2283.1824666983034,3124.22518947067,2042.1183731316023,3733.348783820831,1982.2782545836733,2084.2586308058353]

In [30]:
diffindex = zip difflist [0..]

In [31]:
take 5 $ sortBy (\a b -> if fst a < fst b then LT else GT) diffindex

[(0.0,10),(1684.2193564396582,186),(1700.377045033831,179),(1745.3169462446886,105),(1748.1366837709609,64)]

In [32]:
movieRecommend i a b = m ! a ! i == 0 && m ! b ! i >= 4

In [33]:
rec = map (\i -> movieRecommend i 10 186) [0..1681]

In [34]:
filter fst (zip rec [1..])

[(True,23),(True,64),(True,65),(True,116),(True,137),(True,179),(True,186),(True,197),(True,209),(True,210),(True,214),(True,275),(True,462),(True,651),(True,694),(True,792),(True,1065)]

In [35]:
quickQuery db "SELECT title FROM items WHERE items.movieid=23" []

[[SqlByteString "Taxi Driver (1976)"]]

In [36]:
quickQuery db "SELECT title FROM items WHERE items.movieid=64" []

[[SqlByteString "Shawshank Redemption, The (1994)"]]

In [37]:
quickQuery db "SELECT title FROM items WHERE items.movieid=65" []

[[SqlByteString "What's Eating Gilbert Grape (1993)"]]